In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import csv
import openpyxl
import time
import pandas as pd

# Companies-based

In [7]:
# Load the Excel spreadsheet containing the list of company names
wb = openpyxl.load_workbook('/Users/andreamaffei/Desktop/Masters/Classes/Spring Semester/Programming/Project/data/executives_clean.xlsx')
sheet = wb.active

# Specify the starting and ending rows (inclusive)
start_row = 2
end_row = 5

# Create a new workbook to store the results
result_wb = openpyxl.Workbook()
result_ws = result_wb.active
result_ws.append(['Company Name', 'Staff Name', 'Title', 'Company Address'])

# Open a new Chrome browser window and navigate to the website
driver = webdriver.Chrome()
driver.get('https://corp.sec.state.ma.us/corpweb/CorpSearch/CorpSearch.aspx')

# Find the input field for company names and loop through each company in the specified range of rows
input_field = driver.find_element(By.CSS_SELECTOR, '#MainContent_txtEntityName')
for row in sheet.iter_rows(min_row=start_row, max_row=end_row, max_col=1):
    company_name = row[0].value
    input_field.clear()
    input_field.send_keys(company_name)
    input_field.send_keys(Keys.RETURN)

    # Wait for the search results page to load
    driver.implicitly_wait(5)

    # Click on the first link to go to the company details page
    link = driver.find_element(By.CSS_SELECTOR,'#MainContent_SearchControl_grdSearchResultsEntity > tbody > tr:nth-child(2) > th > a')
    link.click()

    # Wait for the company details page to load
    driver.implicitly_wait(5)

    # Get the company name from the details page
    company_name = driver.find_element(By.CSS_SELECTOR,'#MainContent_lblEntityNameHeader').text.strip()

    # Get the table of staff names and save it to the result workbook
    try:
        table = driver.find_element(By.CSS_SELECTOR,'#MainContent_grdOfficers')
        rows = table.find_elements_by_tag_name('tr')
        for row in rows[1:]:
            cols = row.find_elements_by_tag_name('td')
            th_col = row.find_element_by_tag_name('th')
            title = th_col.text
            staff_name = cols[0].text
            address = cols[1].text
            result_ws.append([company_name, staff_name, title, address])
    except:
        print(f"An error occurred while processing {company_name}. Moving on to the next company.")

    # Go back to the search page to search for the next company
    driver.get('https://corp.sec.state.ma.us/corpweb/CorpSearch/CorpSearch.aspx')
    input_field = driver.find_element(By.CSS_SELECTOR,'#MainContent_txtEntityName')

# Save the result workbook
result_wb.save('results.xlsx')

# Close the browser window
driver.quit()

An error occurred while processing A - 1 FOOD MART CORPORATION. Moving on to the next company.
An error occurred while processing A CLASS QUALITY INC. Moving on to the next company.
An error occurred while processing A CLEANER TOMORROW INC. Moving on to the next company.
An error occurred while processing A - LIST HOMES REAL ESTATE, INC.. Moving on to the next company.


In [6]:
# CLEAN WEBSCRAPING 1

# Load the Excel file
df = pd.read_excel('results.xlsx')

# Split the Staff Name column into first_name and last_name columns
df['first_name'] = df['Staff Name'].str.split(' ').str[0]
df['last_name'] = df['Staff Name'].str.split(' ').str[-1]
df = df[['first_name', 'last_name', 'Staff Name', 'Title', 'Address']]

# Save the modified DataFrame as a new Excel file
df.to_excel('results-cleaned.xlsx', index=False)


KeyError: "['Address'] not in index"

# Executives-based

In [24]:
import openpyxl
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# Define the URL of the website and the path to the Excel file
url = "https://corp.sec.state.ma.us/corpweb/CorpSearch/CorpSearch.aspx"
excel_file = "executives.xlsx"
output_file = "results-executives.csv"

# Set up the web driver
driver = webdriver.Chrome()

# Read the Excel file and search for each name on the website
workbook = openpyxl.load_workbook(excel_file)
worksheet = workbook.active
header_row = ["first_name", "middle_name", "last_name", "results"]

with open(output_file, "w", newline="") as out_f:
    writer = csv.writer(out_f)
    writer.writerow(header_row)  # write header row

    for row in worksheet.iter_rows(min_row=2, min_col=1, max_col=3):
        first_name, middle_name, last_name = [cell.value for cell in row]

        # Navigate to the website and click on the radio button for individual search
        driver.get(url)
        driver.find_element_by_css_selector("#MainContent_rdoByIndividual").click()
        
        # Wait for the page to load
        #driver.implicitly_wait(15)
        WebDriverWait(driver, 10).until(EC.staleness_of(driver.find_element_by_css_selector("#MainContent_tblByIndividual")))

        # Enter the first name, middle name, and last name in the appropriate fields
        driver.find_element_by_css_selector("#MainContent_txtFirstName").send_keys(first_name)
        driver.find_element_by_css_selector("#MainContent_txtMiddleName").send_keys(middle_name)
        driver.find_element_by_css_selector("#MainContent_txtLastName").send_keys(last_name)

        # Click on the search button
        driver.find_element_by_css_selector("#MainContent_btnSearch").click()

        # Wait for the search results to load and extract them
        try:
            WebDriverWait(driver, 10).until(EC.staleness_of(driver.find_element_by_css_selector("#MainContent_SearchControl_tblGrid1")))
            search_results = driver.find_elements_by_css_selector("#MainContent_SearchControl_tblGrid1 tr")
            results = []
            for result in search_results:
                # process the search results as needed
                results.append(result.text)
            writer.writerow([first_name, middle_name, last_name, "\n".join(results)])
        except:
            # handle any errors that occur while waiting for the search results to load
            writer.writerow([first_name, middle_name, last_name, "Error"])
        
        # Return to the original page to perform a new search
        driver.get(url)

# Close the web driver
driver.quit()


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.49)


In [36]:
# WEBSCRAPIN 2 - No middle name

# Define the URL of the website and the path to the Excel file
url = "https://corp.sec.state.ma.us/corpweb/CorpSearch/CorpSearch.aspx"
excel_file = "results-cleaned.xlsx"
output_file = "results-executives.csv"

# Set up the web driver
driver = webdriver.Chrome()

# Read the Excel file and search for each name on the website
workbook = openpyxl.load_workbook(excel_file)
worksheet = workbook.active
header_row = ["first_name", "last_name", "results"]

with open(output_file, "w", newline="") as out_f:
    writer = csv.writer(out_f)
    writer.writerow(header_row)  # write header row

    for row in worksheet.iter_rows(min_row=2, min_col=1, max_col=2):
        first_name, last_name = [cell.value for cell in row]

        # Navigate to the website and click on the radio button for individual search
        driver.get(url)
        driver.find_element_by_css_selector("#MainContent_rdoByIndividual").click()
        
        # Wait for the page to load
        WebDriverWait(driver, 10).until(EC.staleness_of(driver.find_element_by_css_selector("#MainContent_tblByIndividual")))

        # Enter the first name and last name in the appropriate fields
        driver.find_element_by_css_selector("#MainContent_txtFirstName").send_keys(first_name)
        driver.find_element_by_css_selector("#MainContent_txtLastName").send_keys(last_name)

        # Click on the search button
        driver.find_element_by_css_selector("#MainContent_btnSearch").click()

        # Wait for the search results to load and extract them
        try:
            WebDriverWait(driver, 10).until(EC.staleness_of(driver.find_element_by_css_selector("#MainContent_SearchControl_tblGrid1")))
            search_results = driver.find_elements_by_css_selector("#MainContent_SearchControl_tblGrid1 tr")
            results = []
            for result in search_results:
                # process the search results as needed
                results.append(result.text)
            writer.writerow([first_name, last_name, "\n".join(results)])
        except:
            # handle any errors that occur while waiting for the search results to load
            writer.writerow([first_name, last_name, "Error"])
        
        # Return to the original page to perform a new search
        driver.get(url)

# Close the web driver
driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#MainContent_rdoByIndividual"}
  (Session info: chrome=112.0.5615.49)
